In [46]:
import math
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
import sklearn.linear_model as skl
model = skl.LinearRegression()
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from math import sqrt
import xgboost as xgb

# loading training/testing data
trainDataframe = pd.read_csv("train.csv")
testframe = pd.read_csv("test.csv")

In [47]:
trainDataframe = trainDataframe.set_index("id")
trainDataframe_new = trainDataframe
trainDataframe_new['is_free']=trainDataframe_new['is_free'].astype('int')
#trainDataframe_new=trainDataframe.dropna(how='any', axis = 0)

genres = trainDataframe_new["genres"].str.get_dummies(",").add_prefix('genres_')
trainDataframe_new = pd.concat([trainDataframe_new, genres], axis = 1) 

categories = trainDataframe_new["categories"].str.get_dummies(",").add_prefix('categories_')
trainDataframe_new = pd.concat([trainDataframe_new, categories], axis = 1)

#tags = trainDataframe_new["tags"].str.get_dummies(",").add_prefix('tags_')
#trainDataframe_new = pd.concat([trainDataframe_new, tags], axis = 1)

trainDataframe_new['purchase_date']= pd.to_datetime(trainDataframe_new['purchase_date']) 
trainDataframe_new['release_date']= pd.to_datetime(trainDataframe_new['release_date']) 
trainDataframe_new['time_diff'] = (trainDataframe_new['purchase_date']-trainDataframe_new['release_date']).astype('timedelta64[D]')
#print(trainDataframe_new['time_diff'].mean(axis = 0) )
#put mean into nan record
trainDataframe_new['time_diff']=trainDataframe_new['time_diff'].fillna(value=trainDataframe_new['time_diff'].mean(axis = 0) ) 

#print(testframe_new['time_diff'])
trainDataframe_new['total_positive_reviews']=trainDataframe_new['total_positive_reviews'].fillna(0) 
trainDataframe_new['total_negative_reviews']=trainDataframe_new['total_negative_reviews'].fillna(0) 
#print(trainDataframe_new['total_positive_reviews'])


trainDataframe_new.drop(["purchase_date"], axis = 1 ,inplace=True)
trainDataframe_new.drop(["release_date"], axis = 1,inplace=True)

trainDataframe_new.drop(["genres","categories","tags"], axis = 1,inplace=True)
#df1 = trainDataframe_new[pd.isnull(trainDataframe_new).any(axis=1)]
#print(df1)

In [48]:
testframe = testframe.set_index("id")
testframe_new=testframe
testframe_new['is_free']=testframe_new['is_free'].astype('int')

genres_test = testframe_new["genres"].str.get_dummies(",").add_prefix('genres_')
testframe_new = pd.concat([testframe_new, genres_test], axis = 1) 

categories_test = testframe_new["categories"].str.get_dummies(",").add_prefix('categories_')
testframe_new = pd.concat([testframe_new, categories_test], axis = 1) 

#tags_test = testframe_new["tags"].str.get_dummies(",").add_prefix('tags_')
#testframe_new = pd.concat([testframe_new, tags_test], axis = 1) 

testframe_new['purchase_date']= pd.to_datetime(testframe_new['purchase_date']) 
testframe_new['release_date']= pd.to_datetime(testframe_new['release_date']) 
testframe_new['time_diff'] = (testframe_new['purchase_date']-testframe_new['release_date']).astype('timedelta64[D]')
 
testframe_new['time_diff'] = testframe_new['time_diff'].fillna(value=testframe_new['time_diff'].mean(axis = 0) ) 

testframe_new['total_positive_reviews']=testframe_new['total_positive_reviews'].fillna(0) 
testframe_new['total_negative_reviews']=testframe_new['total_negative_reviews'].fillna(0) 


testframe_new.drop(["purchase_date"], axis = 1 ,inplace=True)
testframe_new.drop(["release_date"], axis = 1,inplace=True)

testframe_new.drop(["genres","categories","tags"], axis = 1,inplace=True)

## find null value
#df1 = testframe_new[pd.isnull(testframe_new).any(axis=1)]
#print(df1)

In [49]:
trainDataframe_new = trainDataframe_new.loc[:,~trainDataframe_new.columns.duplicated()]
testframe_new = testframe_new.loc[:,~testframe_new.columns.duplicated()]
#find common fields
cols = list(trainDataframe_new.columns.intersection(testframe_new.columns))
#12,45
#print(len(cols))

In [50]:
##LinearRegression
model = skl.LinearRegression()
model.fit(trainDataframe_new[cols], trainDataframe_new["playtime_forever"])
predict_y = model.predict(trainDataframe_new[cols])
#predict_y = model.predict(testframe_new[cols])
#df = pd.DataFrame({'playtime_forever':predict_y})
print("LinearRegression - sqrt mean sqrt err: %.3f"
      % sqrt(mean_squared_error(trainDataframe_new["playtime_forever"], predict_y)))
#df[df < 0] = 0
#print(df)
#df.to_csv('Result.csv')
predict_y = model.predict(testframe_new[cols])
df = pd.DataFrame({'playtime_forever':predict_y})
df[df < 0] = 0
#print(df)
df.to_csv('Result_linear.csv')

##RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_features=47, min_samples_split=4, n_estimators=50, min_samples_leaf=2)
model.fit(trainDataframe_new[cols], trainDataframe_new["playtime_forever"])
predict_y = model.predict(trainDataframe_new[cols])
#df = pd.DataFrame({'playtime_forever':predict_y})
print("RandomForestRegressor -sqrt mean sqrt err: %.3f"
      % sqrt(mean_squared_error(trainDataframe_new["playtime_forever"], predict_y)))
predict_y = model.predict(testframe_new[cols])
df = pd.DataFrame({'playtime_forever':predict_y})
df[df < 0] = 0
df.to_csv('Result_rf.csv')


##KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=6)
model.fit(trainDataframe_new[cols], trainDataframe_new["playtime_forever"])
predict_y = model.predict(trainDataframe_new[cols])
print("KNeighborsRegressor -sqrt mean sqrt err: %.3f"
      % sqrt(mean_squared_error(trainDataframe_new["playtime_forever"], predict_y)))

#XGBRegressor
xgb_model = xgb.XGBRegressor()
xgb_model.fit(trainDataframe_new[cols], trainDataframe_new["playtime_forever"])
predict_y = xgb_model.predict(trainDataframe_new[cols])
print("XGBRegressor -sqrt mean sqrt err: %.3f"
      % sqrt(mean_squared_error(trainDataframe_new["playtime_forever"], predict_y)))
predict_y = model.predict(testframe_new[cols])
df = pd.DataFrame({'playtime_forever':predict_y})
df[df < 0] = 0
df.to_csv('Result_xgb.csv')


#boosting
gb = GradientBoostingRegressor(loss='ls', learning_rate=0.0001, n_estimators=50, max_features='log2', min_samples_split=2, max_depth=1)
gb.fit(trainDataframe_new[cols], trainDataframe_new["playtime_forever"])
predict_y = gb.predict(trainDataframe_new[cols])
print("boosting -sqrt mean sqrt err: %.3f"
      % sqrt(mean_squared_error(trainDataframe_new["playtime_forever"], predict_y)))
predict_y = gb.predict(testframe_new[cols])
df = pd.DataFrame({'playtime_forever':predict_y})
df[df < 0] = 0
df.to_csv('Result_boosting.csv')

LinearRegression - sqrt mean sqrt err: 9.119
RandomForestRegressor -sqrt mean sqrt err: 5.789
KNeighborsRegressor -sqrt mean sqrt err: 9.658
[10:32:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor -sqrt mean sqrt err: 2.648
boosting -sqrt mean sqrt err: 11.195


/Users/venusleung/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/venusleung/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
